So what we are gonna do is develop a dyanmic embeddings model that will choose which element to use on the fly based off the query and the document retrieved 

In [ ]:
# !pip install sentence-transformers
# !pip install transformers



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# # !pip install duckduckgo-search
# !pip install fake-useragent

# !pip install requests


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


so we are using duckduckgo_Search and the for 403 errors we are using fake agents and complete headers so we get few 403 errors and get the 5 json dump from all this

In [34]:
import json
import random
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

class DuckDuckGoWebLoader:
    def __init__(self, query, num_results=5):
        self.query = query
        self.num_results = num_results

    def load(self):
        ddgs = DDGS()
        # Request extra search results to compensate for potential errors.
        extra_multiplier = 2
        search_results = ddgs.text(self.query, max_results=self.num_results * extra_multiplier)
        documents = []
        
        for result in search_results:
            if len(documents) >= self.num_results:
                break  # Stop once we have the desired number of valid documents

            title = result.get("title", "No Title")
            link = result.get("href")
            description = result.get("body", "")
            description = " ".join(description.split())  # Normalize spacing

            if not link:
                continue

            try:
                headers = self._generate_headers()
                response = requests.get(link, headers=headers, timeout=10)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.text, "html.parser")
                content = soup.get_text(separator="\n").strip()
                content = " ".join(content.split())  # Normalize spacing
                date = "Unknown"  # Placeholder for date extraction
                
                documents.append({
                    "title": title,
                    "link": link,
                    "description": description,
                    "content": content,
                    "date": date
                })
            except Exception as e:
                print(f"Error fetching {link}: {e}")

        return documents

    def _generate_headers(self):
        # Complete set of headers with a fixed User-Agent and other common headers.
        return {
            "authority": "www.google.com",
            "accept": ("text/html,application/xhtml+xml,application/xml;q=0.9,"
                       "image/avif,image/webp,image/apng,*/*;q=0.8,"
                       "application/signed-exchange;v=b3;q=0.7"),
            "accept-language": "en-US,en;q=0.9",
            "cache-control": "max-age=0",
            "user-agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                           "AppleWebKit/537.36 (KHTML, like Gecko) "
                           "Chrome/129.0.0.0 Safari/537.36")
        }




In [35]:

query = "latest technology trends"
loader = DuckDuckGoWebLoader(query, num_results=5)
docs = loader.load()
json_output = json.dumps(docs, ensure_ascii=False, indent=2)
print(json_output)

Error fetching https://www.gartner.com/en/articles/top-technology-trends-2025: 403 Client Error: Forbidden for url: https://www.gartner.com/en/articles/top-technology-trends-2025
Error fetching https://www.weforum.org/stories/2024/06/top-10-emerging-technologies-of-2024-impact-world/: 403 Client Error: Forbidden for url: https://www.weforum.org/stories/2024/06/top-10-emerging-technologies-of-2024-impact-world/
Error fetching https://technologymagazine.com/articles/top-10-trends-of-2025: 403 Client Error: Forbidden for url: https://technologymagazine.com/articles/top-10-trends-of-2025
[
  {
    "title": "25 New Technology Trends for 2025 | Emerging Technologies 2025",
    "link": "https://www.simplilearn.com/top-technology-trends-and-jobs-article",
    "description": "1. Generative AI. Starting the list of new technology trends with the talk of the town, gen-AI! Generative AI is set to dominate as a key technology trend in 2025, reshaping industries through its ability to create highly 

In [39]:
print("Testing Math embedding model (math-similarity/Bert-MLM_arXiv-MP-class_zbMath):")
try:
    from sentence_transformers import SentenceTransformer
    def load_math_model():
        model_name = 'math-similarity/Bert-MLM_arXiv-MP-class_zbMath'
        
        model = SentenceTransformer(model_name)

       
        print("Math embedding model loaded successfully.")
        
        return model

    math_model= load_math_model()
except Exception as e:
    print("Error loading Math embedding model:", e)

Testing Math embedding model (math-similarity/Bert-MLM_arXiv-MP-class_zbMath):


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\kulka\.conda\envs\QF_lama\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kulka\.cache\huggingface\hub\models--math-similarity--Bert-MLM_arXiv-MP-class_zbMath. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.57k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Math embedding model loaded successfully.


In [ ]:
# Test script to load multiple embedding models


print("\nTesting CodeBERT embedding model for programming queries:")
try:
    from transformers import AutoModel, AutoTokenizer
    def load_codebert():
        model_name = "microsoft/codebert-base"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        print("CodeBERT embedding model loaded successfully.")
        return model, tokenizer
    codebert_model, codebert_tokenizer = load_codebert()
except Exception as e:
    print("Error loading CodeBERT:", e)

print("\nTesting SciBERT embedding model for scientific queries:")
try:
    from transformers import AutoModel, AutoTokenizer
    def load_scibert():
        model_name = "allenai/scibert_scivocab_uncased"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        print("SciBERT embedding model loaded successfully.")
        return model, tokenizer
    scibert_model, scibert_tokenizer = load_scibert()
except Exception as e:
    print("Error loading SciBERT:", e)

print("\nTesting MedEmbed (Bio_ClinicalBERT) for medical queries:")
try:
    from transformers import AutoModel, AutoTokenizer
    def load_medembed():
        model_name = "emilyalsentzer/Bio_ClinicalBERT"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        print("MedEmbed (Bio_ClinicalBERT) loaded successfully.")
        return model, tokenizer
    medembed_model, medembed_tokenizer = load_medembed()
except Exception as e:
    print("Error loading MedEmbed:", e)

print("\nTesting Sentence Transformers (all-MiniLM-L6-v2) for general knowledge queries:")
try:
    from sentence_transformers import SentenceTransformer
    def load_sentence_transformer():
        model_name = "all-MiniLM-L6-v2"
        model = SentenceTransformer(model_name)
        print("Sentence Transformers (all-MiniLM-L6-v2) loaded successfully.")
        return model
    sentence_transformer_model = load_sentence_transformer()
except Exception as e:
    print("Error loading Sentence Transformer:", e)


Testing Tangent-CFT embedding model (simulated):
Tangent-CFT embedding model loaded successfully (simulated).

Testing CodeBERT embedding model for programming queries:


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\kulka\.conda\envs\QF_lama\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kulka\.cache\huggingface\hub\models--microsoft--codebert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

MedEmbed (Bio_ClinicalBERT) loaded successfully.

Testing Sentence Transformers (all-MiniLM-L6-v2) for general knowledge queries:


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]


Sentence Transformers (all-MiniLM-L6-v2) loaded successfully.
